# Notebook 2 - SQL Analysis
**Objectif :** Effectuer des agrégations exploratoires sur les données originales (non transformées) via Spark SQL.

In [1]:
# 🔹 Étape 1 : Initialisation Spark
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, stddev

spark = SparkSession.builder \
    .appName("Fraud-SQL") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/05 01:01:35 WARN Utils: Your hostname, TUF-GAMING-FX504GD, resolves to a loopback address: 127.0.1.1; using 192.168.1.145 instead (on interface wlo1)
26/01/05 01:01:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/05 01:01:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 🔹 Étape 2 : Charger le dataset CSV original et créer la vue temporaire (CORRECTION)
# Nous chargeons le CSV pour garantir la présence des colonnes 'Amount' et 'Class' nécessaires à l'analyse exploratoire.
train_df = spark.read.csv("hdfs:///user/hadoop/BigDataFraude_ML-GraphX/creditcard.csv", header=True, inferSchema=True)
train_df.createOrReplaceTempView("train_transactions")
train_df.printSchema()

[Stage 1:>                                                        (0 + 12) / 12]

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

26/01/05 01:01:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
# 🔹 Étape 3 : Nombre de transactions par classe
print("--- Nombre de transactions par classe ---")
# 'Class' est résolue car nous chargeons le CSV original
df_class_count = spark.sql("SELECT Class AS label, COUNT(*) AS count FROM train_transactions GROUP BY Class")
df_class_count.show()

--- Nombre de transactions par classe ---


[Stage 2:>                                                        (0 + 12) / 12]

+-----+------+
|label| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



In [4]:
# 🔹 Étape 4 : Montant moyen et Écart-type par classe
print("\n--- Montant moyen et Écart-type par classe ---")
# 'Amount' est résolue et permet le calcul de la moyenne et de l'écart-type
df_avg_amount = spark.sql("""
    SELECT 
        Class AS label, 
        AVG(Amount) AS avg_amount,
        STDDEV(Amount) AS stddev_amount
    FROM train_transactions 
    GROUP BY Class
""")
df_avg_amount.show()


--- Montant moyen et Écart-type par classe ---
+-----+-----------------+------------------+
|label|       avg_amount|     stddev_amount|
+-----+-----------------+------------------+
|    1|122.2113211382114| 256.6832882977121|
|    0|88.29102242231271|250.10509222589235|
+-----+-----------------+------------------+

